In [1]:
import os, gc, torch, PIL, pickle,json,io

import pandas as pd
from typing import List
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from utils.plot import plot_losses, plot_train_val_evaluators
from PIL import Image
from models.load import get_model_label, get_model_name, get_current_epoch, get_dataset_label, get_trained_model

from data.constants import DEFAULT_REFLACX_LABEL_COLS

import utils.print as print_f
from utils.init import reproducibility, clean_memory_get_device

# from models.load import TrainedModels
## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [2]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [3]:
from enum import Enum

class TrainedModels(Enum):
    # mobilenet_baseline = "val_lesion-detection_ap_0_1655_test_lesion-detection_ap_0_1648_epoch50_03-15-2023 16-43-54_lesion_dsetection_baseline_mobilenet"  # mobilenet baseline
    # mobilenet_with_fix = "val_lesion-detection_ap_0_1918_test_lesion-detection_ap_0_1903_epoch16_03-16-2023 11-34-10_lesion_dsetection_with_fixation_mobilenet"
    # resnet18_baseline = "val_lesion-detection_ap_0_1973_test_lesion-detection_ap_0_2010_epoch22_03-16-2023 19-44-55_lesion_dsetection_baseline_resnet"
    # resnet18_with_fix = "val_lesion-detection_ap_0_1951_test_lesion-detection_ap_0_2195_epoch12_03-17-2023 00-31-54_lesion_dsetection_with_fixation_resnet"
    # densenet161_baseline = "val_lesion-detection_ap_0_1990_test_lesion-detection_ap_0_2085_epoch5_03-17-2023 08-53-33_lesion_dsetection_baseline_densenet161"
    # densenet161_with_fix = "val_lesion-detection_ap_0_2120_test_lesion-detection_ap_0_2104_epoch12_03-17-2023 18-36-01_lesion_dsetection_with_fixation_densenet161"
    # efficientnet_b5_baseline = "val_lesion-detection_ap_0_1898_test_lesion-detection_ap_0_2055_epoch5_03-17-2023 23-30-57_lesion_dsetection_baseline_efficientnet_b5"
    # efficientnet_b5_with_fix = "val_lesion-detection_ap_0_2117_test_lesion-detection_ap_0_2190_epoch8_03-18-2023 12-29-20_lesion_dsetection_with_fixation_efficientnet_b5"
    # efficientnet_b0_baseline = "val_lesion-detection_ap_0_1934_test_lesion-detection_ap_0_1858_epoch10_03-18-2023 23-50-47_lesion_dsetection_baseline_efficientnet_b0"
    # efficientnet_b0_with_fix = "val_lesion-detection_ap_0_2191_test_lesion-detection_ap_0_2162_epoch10_03-18-2023 19-38-11_lesion_dsetection_with_fixation_efficientnet_b0"
    convnext_base_with_fix = "val_lesion-detection_ap_0_2610_test_lesion-detection_ap_0_2548_epoch22_03-22-2023 02-55-37_lesion_dsetection_with_fixation_convnext_base"
    convnext_base_baseline = "val_lesion-detection_ap_0_2426_test_lesion-detection_ap_0_2325_epoch20_03-22-2023 11-53-53_lesion_dsetection_baseline_convnext_base"
    convnext_base_with_fix_silent = "val_lesion-detection_ap_0_2405_test_lesion-detection_ap_0_2543_epoch19_03-24-2023 13-57-29_lesion_dsetection_with_fixation_convnext_base_silent_report"
    convnext_base_with_fix_full = "val_lesion-detection_ap_0_2602_test_lesion-detection_ap_0_2499_epoch22_03-24-2023 04-42-21_lesion_dsetection_with_fixation_convnext_base_full_report"
    # vgg16_with_fix = "val_lesion-detection_ap_0_2301_test_lesion-detection_ap_0_2186_epoch22_03-20-2023 19-26-02_lesion_dsetection_with_fixation_vgg16"
    # vgg16_baseline = "val_lesion-detection_ap_0_2113_test_lesion-detection_ap_0_2068_epoch12_03-21-2023 00-45-24_lesion_dsetection_baseline_vgg16"
    # regnet_y_8gf_with_fix = "val_lesion-detection_ap_0_2267_test_lesion-detection_ap_0_2029_epoch12_03-21-2023 11-28-48_lesion_dsetection_with_fixation_regnet_y_8gf"
    # regnet_y_8gf_baseline = "val_lesion-detection_ap_0_1883_test_lesion-detection_ap_0_1658_epoch13_03-21-2023 15-22-32_lesion_dsetection_baseline_regnet_y_8gf"

In [4]:
naming_map = {
    # TrainedModels.mobilenet_baseline: "mobilenet_baseline",
    # TrainedModels.mobilenet_with_fix: "mobilenet_with_fix",
    # TrainedModels.resnet18_baseline: "resnet18_baseline",
    # TrainedModels.resnet18_with_fix: "resnet18_with_fix",
    # TrainedModels.densenet161_baseline: "densenet161_baseline",
    # TrainedModels.densenet161_with_fix: "densenet161_with_fix",
    # TrainedModels.efficientnet_b5_baseline: "efficientnet_b5_baseline",
    # TrainedModels.efficientnet_b5_with_fix: "efficientnet_b5_with_fix",
    # TrainedModels.efficientnet_b0_baseline: "efficientnet_b0_baseline",
    # TrainedModels.efficientnet_b0_with_fix: "efficientnet_b0_with_fix",
    TrainedModels.convnext_base_with_fix: "convnext_base_with_fix",
    TrainedModels.convnext_base_baseline: "convnext_base_baseline",
    TrainedModels.convnext_base_with_fix_silent: "convnext_base_with_fix_silent",
    TrainedModels.convnext_base_with_fix_full:"convnext_base_with_fix_full",
    # TrainedModels.vgg16_with_fix: "vgg16_with_fix",
    # TrainedModels.vgg16_baseline: "vgg16_baseline",
    # TrainedModels.regnet_y_8gf_with_fix: "regnet_y_8gf_with_fix",
    # TrainedModels.regnet_y_8gf_baseline: "regnet_y_8gf_baseline",
}


In [5]:
score_thrs="0.05"

In [6]:
eval_ss = []

for select_model in TrainedModels:
    model_path = select_model.value
    for l in DEFAULT_REFLACX_LABEL_COLS + ["all"]:
        eval_df = pd.read_csv(
            os.path.join(
                "./eval_results",
                f"{model_path}_{l}_{score_thrs}.csv",
            ),
            index_col=0,
        )
        eval_s = eval_df.iloc[0]
        eval_s["model"] = naming_map[select_model]
        eval_s["lesion"] = l
        eval_s["score_threshold"] = score_thrs
        states = json.loads(eval_s["coco_states"])
        for s in states:
            eval_s[s["p_title_str"]] = s["mean_s"]
        eval_ss.append(eval_s)


In [7]:
eval_df = pd.DataFrame(eval_ss)

In [8]:
eval_df = eval_df[[
    "model",
    "lesion",
    "num_tps",
    "num_fps",
    "num_fns",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=  5 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]",
    " Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 30 ]",
    " Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 10 ]",
    " Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 10 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  5 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ]",
    " Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 30 ]",
]]


In [9]:
eval_df = eval_df.rename(columns={"num_tps": "#TP @[ IoU=0.50      | area=   all | maxDets= 30 ]", "num_fps": "#FP @[ IoU=0.50      | area=   all | maxDets= 30 ]", "num_fns":"#FN @[ IoU=0.50      | area=   all | maxDets= 30 ]"})

In [10]:
eval_df

,model,lesion,#TP @[ IoU=0.50 | area= all | maxDets= 30 ],#FP @[ IoU=0.50 | area= all | maxDets= 30 ],#FN @[ IoU=0.50 | area= all | maxDets= 30 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ],Average Precision (AP) @[ IoU=0.50 | area= all | maxDets= 10 ],Average Precision (AP) @[ IoU=0.75 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 1 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 5 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 10 ],Average Recall (AR) @[ IoU=0.50:0.95 | area= all | maxDets= 30 ]
0,convnext_base_with_fix,Pulmonary edema,161,1495,38,0.031885,0.112155,0.129915,0.135403,0.227878,0.115770,0.131156,0.469849,0.608543,0.663317
0,convnext_base_with_fix,Enlarged cardiac silhouette,209,1021,68,0.175557,0.268562,0.276435,0.276809,0.390698,0.292780,0.282310,0.563899,0.611191,0.614079
0,convnext_base_with_fix,Consolidation,213,736,124,0.095747,0.151104,0.156502,0.157231,0.287906,0.129155,0.201780,0.425816,0.459941,0.464392
0,convnext_base_with_fix,Atelectasis,223,1113,94,0.053620,0.111548,0.114922,0.115275,0.220843,0.101991,0.180126,0.459306,0.491798,0.494953
0,convnext_base_with_fix,Pleural abnormality,142,417,262,0.068346,0.068427,0.067797,0.068304,0.140384,0.057208,0.131931,0.208911,0.216089,0.219802
0,convnext_base_with_fix,all,950,4772,584,0.085205,0.141252,0.148674,0.150011,0.253627,0.139889,0.186691,0.424594,0.477909,0.491487
0,convnext_base_baseline,Pulmonary edema,146,1397,53,0.027234,0.096848,0.108737,0.110281,0.194531,0.099858,0.126131,0.445226,0.557286,0.578894
0,convnext_base_baseline,Enlarged cardiac silhouette,194,967,83,0.180087,0.257155,0.265189,0.265149,0.393081,0.302076,0.263538,0.504332,0.546931,0.546931
0,convnext_base_baseline,Consolidation,263,1975,74,0.108269,0.152597,0.161591,0.164357,0.269701,0.138682,0.207418,0.486647,0.584273,0.618991
0,convnext_base_baseline,Atelectasis,135,468,182,0.047997,0.072488,0.073006,0.073006,0.149585,0.055628,0.153628,0.282019,0.284227,0.284227


In [11]:
eval_df.to_csv("evaluation.csv")